In [265]:
from torch.autograd import Variable
from torch.autograd import grad
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
import torch
import copy
import time

dtype=torch.float64
#torch.set_default_dtype(dtype) 

In [266]:
class mySin(torch.nn.Module):
    """Sine activation function as a PyTorch Module 
    (Base class for all neural network modules.)"""
    @staticmethod
    def forward(input):
        return torch.sin(input)

def dfx(x,f):   
    """ Derivatives with Auto-differention
        -----------------------------------
        x: variable
        f: function 
        --------------------------------"""
    return grad([f], [x], grad_outputs=torch.ones(x.shape, dtype=dtype), create_graph=True)[0]

def QNM_SR31(ξ, ψR, ψI, ωR, ωI, M, s, l):
    """ Loss function associated with the differential equation
        (Radial perturbation of Asymptotically Flat Schwarschild 
        in finite domain)
        --------------------------------------------------------
        ξ  : variable domain
        ψR : incognite function (real)
        ψI : incognite function (frequency)
        ωR : eigenvalue (real frecuency)
        ωI : eigenvalue (imaginary frequency)
        M  : mass
        s  : spin of the perturbation
        l  : azimutal number
        ---------------------------------------------------- """
    ψR_dξ = dfx(ξ,ψR)    #First derivative  (real)
    ψR_ddξ= dfx(ξ,ψR_dξ) #Second derivative (real)
    ψI_dξ = dfx(ξ,ψI)    #First derivative  (imaginary)
    ψI_ddξ= dfx(ξ,ψI_dξ) #Second derivative (imaginary)
    #Complex frequency
    ω = ωR - ωI*1j 
    #Auxiliar variables
    λ0 = (4j*M*ω*(2*(ξ**2) - 4*ξ + 1) - (1 - 3*ξ)*(1 - ξ))
    s0 = (16*(M**2)*(ω**2)*(ξ - 2) - 8j*M*ω*(1 - ξ) + l*(l + 1) + (1 - s**2)*(1 - ξ))
    #ODE
    f = (ξ*((1 - ξ)**2))*(ψR_ddξ + ψI_ddξ*1j) - λ0*(ψR_dξ + ψI_dξ*1j) - s0*(ψR + ψI*1j)
    return ((f.abs()).pow(2)).mean() #Returns loss

In [267]:
class NN(torch.nn.Module):
    """Neural Network with two layers"""
    def __init__(self, D_hid=10): #Elements of the model
        """D_hid: Numeber of hidden layers"""
        #Initialize super class
        super(NN,self).__init__()
        # Define the Activation  
        self.actF = mySin()
        # Define layers
        self.ωRin    = torch.nn.Linear(1,1)           #For the real frequency
        self.ωIin    = torch.nn.Linear(1,1)           #For the imaginary frequency
        self.Lin_1  = torch.nn.Linear(3, D_hid)       #Three inputs (x and ωR,ωI)
        self.Lin_2  = torch.nn.Linear(D_hid, D_hid)   #Fully connected layer
        self.out    = torch.nn.Linear(D_hid, 2)       #One output (ψ(x))

    def forward(self,x):           #What the model does
        In1 = torch.sigmoid(self.ωRin(torch.ones_like(x)))
        In2 = torch.sigmoid(self.ωIin(torch.ones_like(x)*(-1)))
        L1 = self.Lin_1(torch.cat((x,In1,In2),1))
        h1 = self.actF(L1)
        L2 = self.Lin_2(h1)
        h2 = self.actF(L2)
        out = self.out(h2)
        return out, In1, In2     #Returns ψ(x) and ωR, ωI 

In [268]:
def Scan_QNM(x0, xf, NpL, epochs, n_train, lr, explore, tol, new, M, s, l):
    """ Explore the space of eigenvalues to find different solutions to 
        the ODE associated to the quasinormal modes in an Asymptotically 
        Flat Schwarschild Black Hole.
        ------------------------------------------------------------------
        x0       : left boundary
        xf       : right boundary
        NpL      : number of neurons per layer 
        epochs   : number of training epochs
        n_train  : number of points in the domain if training
        lr       : learning rate
        explore  : number of epochs before changing cost function (drive)
        tol      : tolerance criteria for acceptance  of eigenvalue
        new      : criteria for distintion between eigenvalues
        M        : mass
        s        : spin of the perturbation
        l        : azimutal number
        ---------------------------------------------------------------"""
    #Creates neural network
    eigenNN = NN(NpL)    
    #Creates Adam optimizer
    #optimizer = optim.SGD(eigenNN.parameters(), lr=0.01, momentum=0.9)

    optimizer = optim.Adam(eigenNN.parameters(), lr=lr, betas=[0.999, 0.9999]) 
    #optimizer = optim.RMSprop(eigenNN.parameters(), lr=lr, alpha=0.9) 

    #Arrays for saving data
    Total_loss     = []   
    ODE_loss       = []
    NonTriv_loss   = []
    Drive_loss     = []
    FreqRe         = []
    FreqIm         = []
    #Dictionary for saving models with different eigenvalues
    dic = {0:(None, None, None), 1:(None, None, None), 2:(None, None, None)}
    #Discretization of the solution
    x = torch.linspace(x0, xf, n_train).reshape(-1,1)
    x.requires_grad = True
    #Starts training    
    T0   = time.time() #Initial (real) time
    cte  = 0.3         #Initial constant for the drive
    n    = 0           #Eigenvalue conter (starts in zero)
    Llim = 1e20
    for tt in range(epochs):
        loss=0.0
        #Network solutions 
        out, ωRe, ωIm = eigenNN(x)
        ψR = out[:,0].reshape(-1,1)
        ψI = out[:,1].reshape(-1,1)
        λ1 = ωRe[0].data.tolist()[0]
        λ2 = ωIm[0].data.tolist()[0]
        #Saves Eigenvalue given by the network
        FreqRe.append(λ1)
        FreqIm.append(λ2)
        #Losses
        lossODE = QNM_SR31(x,ψR,ψI,ωRe,ωIm,M,s,l); ODE_loss.append(lossODE.detach().numpy())
        lossNT = 1/((((ψR + ψI*1j).abs())**2).mean() + 1e-6); NonTriv_loss.append(lossNT.detach().numpy())
        #lossD  = torch.exp(-1*(ωRe**2 + ωIm**2)+cte).mean(); Drive_loss.append(lossD) 
        lossT = lossODE + lossNT #+ lossD; Total_loss.append(lossT.detach().numpy())  
        '''
        if n > 0:
            #lossNext = ((1 / ((( ωRe - dic[n-1][1] )**2 + ( ωIm - dic[n-1][2]  )**2 + 1e-6))).mean() ).mean()
            lossT += lossNext
        '''
        #Applies backprop
        optimizer.zero_grad()              
        lossT.backward(retain_graph=False) 
        optimizer.step()   
        #Print
        if tt%explore == 0:
            cte += .025
            print(tt,":\t", FreqRe[-1],' -i*', FreqIm[-1],"\t",cte)
            # If there has been convergence to an eigenvalue, saves the model into the dictionary
            if tt > 1000 and np.var(np.array(FreqIm[tt-1000:tt])) < tol: 
                if n == 0:
                    dic[n] = (copy.deepcopy(eigenNN), λ1, λ2)
                    print("Eigenvalue founded!\tFrequency = ",'{0:.4g}'.format(λ1),"- i",
                          '{0:.4g}'.format(λ2), "\tEpoch =",tt,
                          "\tTime = ",'{0:.4g}'.format((time.time()- T0)/60)," min") 
                    n += 1
                #Checks if an eigenvalue has been found before
                elif abs((λ1-dic[n-1][1])/dic[n-1][1]) > new:
                    dic[n] = (copy.deepcopy(eigenNN), λ1, λ2)
                    print("Eigenvalue founded!\tFrequency = ",'{0:.4g}'.format(λ1),"- i",
                          '{0:.4}'.format(λ2), "\tEpoch =",tt,
                          "\tTime = ",'{0:.4g}'.format((time.time()- T0)/60)," min") 
                    n += 1
        #Ends the training if five eigenvalues have been found
        if n == 3:
            break
    #Training time    
    runTime = time.time() - T0  
    #Saves all losses in one object
    loss_histories = (Total_loss, ODE_loss, NonTriv_loss)    
    return dic, loss_histories, FreqRe, FreqIm, runTime

# Training 

### $S_0$

In [269]:
#Seed for reproducibility 
torch.manual_seed(0)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-3, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 0, 0
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.6292397379875183  -i* 0.5217627882957458 	 0.325
1000 :	 0.005349351558834314  -i* 0.024918803945183754 	 0.35000000000000003
2000 :	 0.0010841251350939274  -i* 0.017547305673360825 	 0.37500000000000006
3000 :	 0.0005648413207381964  -i* 0.027063798159360886 	 0.4000000000000001
4000 :	 0.00042886429582722485  -i* 0.06499428302049637 	 0.4250000000000001
5000 :	 0.00038195442175492644  -i* 0.1158103421330452 	 0.4500000000000001
6000 :	 0.0003640225622802973  -i* 0.20258744060993195 	 0.47500000000000014
7000 :	 0.00035694410325959325  -i* 0.3080117404460907 	 0.5000000000000001
8000 :	 0.00035412918077781796  -i* 0.34461700916290283 	 0.5250000000000001
9000 :	 0.0003530272515490651  -i* 0.345729798078537 	 0.5500000000000002
Eigenvalue founded!	Frequency =  0.000353 - i 0.3457 	Epoch = 9000 	Time =  0.895  min
10000 :	 0.0003526219225022942  -i* 0.3527611494064331 	 0.5750000000000002
11000 :	 0.0003524792555253953  -i* 0.3515552580356598 	 0.6000000000000002
12000 :	 0.00035

In [270]:
#Seed for reproducibility 
torch.manual_seed(0)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-3, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 0, 1
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.6292397379875183  -i* 0.5217627882957458 	 0.325
1000 :	 0.287759393453598  -i* 0.013273312710225582 	 0.35000000000000003
2000 :	 0.2869880795478821  -i* 0.008786182850599289 	 0.37500000000000006
3000 :	 0.28243979811668396  -i* 0.014173459261655807 	 0.4000000000000001
4000 :	 0.28931474685668945  -i* 0.05986282229423523 	 0.4250000000000001
5000 :	 0.2928389310836792  -i* 0.1164468303322792 	 0.4500000000000001
6000 :	 0.29100292921066284  -i* 0.0973072201013565 	 0.47500000000000014
7000 :	 0.2927352488040924  -i* 0.09751676023006439 	 0.5000000000000001
8000 :	 0.29463663697242737  -i* 0.09764596819877625 	 0.5250000000000001
9000 :	 0.2933126986026764  -i* 0.09770991653203964 	 0.5500000000000002
Eigenvalue founded!	Frequency =  0.2933 - i 0.09771 	Epoch = 9000 	Time =  1.006  min
10000 :	 0.2926798462867737  -i* 0.0973106175661087 	 0.5750000000000002
11000 :	 0.2926240861415863  -i* 0.09730615466833115 	 0.6000000000000002
12000 :	 0.29273855686187744  -i* 0.09746128320

In [271]:
#Seed for reproducibility 
torch.manual_seed(0)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-3, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 0, 2
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.6292397379875183  -i* 0.5217627882957458 	 0.325
1000 :	 0.49913591146469116  -i* 0.021262628957629204 	 0.35000000000000003
2000 :	 0.47962793707847595  -i* 0.054336290806531906 	 0.37500000000000006
3000 :	 0.481991708278656  -i* 0.08934012800455093 	 0.4000000000000001
4000 :	 0.4851202368736267  -i* 0.09716606140136719 	 0.4250000000000001
5000 :	 0.48454827070236206  -i* 0.09862952679395676 	 0.4500000000000001
6000 :	 0.48516175150871277  -i* 0.09746094793081284 	 0.47500000000000014
7000 :	 0.484538197517395  -i* 0.09708910435438156 	 0.5000000000000001
8000 :	 0.4838503301143646  -i* 0.09668072313070297 	 0.5250000000000001
Eigenvalue founded!	Frequency =  0.4839 - i 0.09668 	Epoch = 8000 	Time =  0.7798  min
9000 :	 0.48362448811531067  -i* 0.09631770104169846 	 0.5500000000000002
10000 :	 0.4837368130683899  -i* 0.09674052149057388 	 0.5750000000000002
11000 :	 0.48353317379951477  -i* 0.09670469164848328 	 0.6000000000000002
12000 :	 0.4835616946220398  -i* 0.09675116

In [272]:
#Seed for reproducibility 
torch.manual_seed(0)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-3, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 0, 3
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.6292397379875183  -i* 0.5217627882957458 	 0.325
1000 :	 0.6931595206260681  -i* 0.03256595507264137 	 0.35000000000000003
2000 :	 0.6793718934059143  -i* 0.1098262295126915 	 0.37500000000000006
3000 :	 0.6722154021263123  -i* 0.10025918483734131 	 0.4000000000000001
4000 :	 0.6706262230873108  -i* 0.0939960852265358 	 0.4250000000000001
5000 :	 0.6723658442497253  -i* 0.09510049968957901 	 0.4500000000000001
6000 :	 0.6745430827140808  -i* 0.09809935837984085 	 0.47500000000000014
7000 :	 0.6756482124328613  -i* 0.09641613066196442 	 0.5000000000000001
8000 :	 0.6754315495491028  -i* 0.09678999334573746 	 0.5250000000000001
9000 :	 0.6751812100410461  -i* 0.09598458558320999 	 0.5500000000000002
10000 :	 0.6753008961677551  -i* 0.09625120460987091 	 0.5750000000000002
Eigenvalue founded!	Frequency =  0.6753 - i 0.09625 	Epoch = 10000 	Time =  0.9829  min
11000 :	 0.675388514995575  -i* 0.09617414325475693 	 0.6000000000000002
12000 :	 0.6754083037376404  -i* 0.0962447971105575

### $S_1$

In [273]:
#Seed for reproducibility 
torch.manual_seed(0)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-3, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 1, 1
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.6292397379875183  -i* 0.5217627882957458 	 0.325
1000 :	 0.13596977293491364  -i* 0.011580424383282661 	 0.35000000000000003
2000 :	 0.21966317296028137  -i* 0.005264439154416323 	 0.37500000000000006
3000 :	 0.25370824337005615  -i* 0.0053680092096328735 	 0.4000000000000001
Eigenvalue founded!	Frequency =  0.2537 - i 0.005368 	Epoch = 3000 	Time =  0.294  min
4000 :	 0.24491381645202637  -i* 0.008210896514356136 	 0.4250000000000001
5000 :	 0.2412748634815216  -i* 0.020887937396764755 	 0.4500000000000001
6000 :	 0.25417467951774597  -i* 0.1251855343580246 	 0.47500000000000014
7000 :	 0.2448672652244568  -i* 0.09640470147132874 	 0.5000000000000001
8000 :	 0.24845847487449646  -i* 0.0910593792796135 	 0.5250000000000001
9000 :	 0.24845252931118011  -i* 0.09441002458333969 	 0.5500000000000002
10000 :	 0.24875958263874054  -i* 0.09272599965333939 	 0.5750000000000002
11000 :	 0.24917402863502502  -i* 0.09227964282035828 	 0.6000000000000002
12000 :	 0.2487328201532364  -i* 0.0

In [274]:
#Seed for reproducibility 
torch.manual_seed(0)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-3, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 1, 2
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.6292397379875183  -i* 0.5217627882957458 	 0.325
1000 :	 0.47762608528137207  -i* 0.017758600413799286 	 0.35000000000000003
2000 :	 0.4627470374107361  -i* 0.03199918195605278 	 0.37500000000000006
3000 :	 0.4592108130455017  -i* 0.12640099227428436 	 0.4000000000000001
4000 :	 0.45669353008270264  -i* 0.10140059143304825 	 0.4250000000000001
5000 :	 0.45850270986557007  -i* 0.0941709503531456 	 0.4500000000000001
6000 :	 0.45709049701690674  -i* 0.09457474946975708 	 0.47500000000000014
7000 :	 0.45787864923477173  -i* 0.09541057795286179 	 0.5000000000000001
8000 :	 0.45807719230651855  -i* 0.09492772072553635 	 0.5250000000000001
Eigenvalue founded!	Frequency =  0.4581 - i 0.09493 	Epoch = 8000 	Time =  0.7663  min
9000 :	 0.45769786834716797  -i* 0.09471381455659866 	 0.5500000000000002
10000 :	 0.457520455121994  -i* 0.09483462572097778 	 0.5750000000000002
11000 :	 0.4575826823711395  -i* 0.09479457885026932 	 0.6000000000000002
12000 :	 0.45758843421936035  -i* 0.0948051

In [275]:
#Seed for reproducibility 
torch.manual_seed(0)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-3, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 1, 3
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.6292397379875183  -i* 0.5217627882957458 	 0.325
1000 :	 0.6422743797302246  -i* 0.014284759759902954 	 0.35000000000000003
2000 :	 0.6751055121421814  -i* 0.02058693952858448 	 0.37500000000000006
3000 :	 0.6732191443443298  -i* 0.1138668954372406 	 0.4000000000000001
4000 :	 0.658397912979126  -i* 0.09935712069272995 	 0.4250000000000001
5000 :	 0.6581891775131226  -i* 0.09376393258571625 	 0.4500000000000001
6000 :	 0.6573377847671509  -i* 0.0960422083735466 	 0.47500000000000014
7000 :	 0.6566640734672546  -i* 0.09520593285560608 	 0.5000000000000001
8000 :	 0.6574623584747314  -i* 0.09572802484035492 	 0.5250000000000001
9000 :	 0.657174289226532  -i* 0.09550043940544128 	 0.5500000000000002
Eigenvalue founded!	Frequency =  0.6572 - i 0.0955 	Epoch = 9000 	Time =  0.8548  min
10000 :	 0.6570064425468445  -i* 0.09536219388246536 	 0.5750000000000002
11000 :	 0.6570661664009094  -i* 0.0955730453133583 	 0.6000000000000002
12000 :	 0.6568772792816162  -i* 0.09570905566215515 	

### $S_2$

In [276]:
#Seed for reproducibility 
torch.manual_seed(0)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-3, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 2, 1
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.6292397379875183  -i* 0.5217627882957458 	 0.325
1000 :	 0.004680831450968981  -i* 0.007085960358381271 	 0.35000000000000003
2000 :	 0.0007702322327531874  -i* 0.0013684885343536735 	 0.37500000000000006
3000 :	 0.0003675525658763945  -i* 0.0006900230655446649 	 0.4000000000000001
Eigenvalue founded!	Frequency =  0.0003676 - i 0.00069 	Epoch = 3000 	Time =  0.2864  min
4000 :	 0.0002686843799892813  -i* 0.0005145756294950843 	 0.4250000000000001
Eigenvalue founded!	Frequency =  0.0002687 - i 0.0005146 	Epoch = 4000 	Time =  0.3836  min
5000 :	 0.0002353488962398842  -i* 0.0004526801058091223 	 0.4500000000000001
Eigenvalue founded!	Frequency =  0.0002353 - i 0.0004527 	Epoch = 5000 	Time =  0.48  min


In [277]:
#Seed for reproducibility 
torch.manual_seed(0)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-3, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 2, 2
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.6292397379875183  -i* 0.5217627882957458 	 0.325
1000 :	 0.3244114816188812  -i* 0.009440543130040169 	 0.35000000000000003
2000 :	 0.3855791687965393  -i* 0.005401184782385826 	 0.37500000000000006
3000 :	 0.37641483545303345  -i* 0.006778310518711805 	 0.4000000000000001
4000 :	 0.3683525025844574  -i* 0.014498892240226269 	 0.4250000000000001
5000 :	 0.37343353033065796  -i* 0.09737817943096161 	 0.4500000000000001
6000 :	 0.3721001446247101  -i* 0.08059768378734589 	 0.47500000000000014
7000 :	 0.37091755867004395  -i* 0.08934067189693451 	 0.5000000000000001
8000 :	 0.3735430836677551  -i* 0.0894952341914177 	 0.5250000000000001
9000 :	 0.3741401433944702  -i* 0.08960264921188354 	 0.5500000000000002
10000 :	 0.3733249306678772  -i* 0.0900590643286705 	 0.5750000000000002
11000 :	 0.3741017282009125  -i* 0.08892413973808289 	 0.6000000000000002
Eigenvalue founded!	Frequency =  0.3741 - i 0.08892 	Epoch = 11000 	Time =  1.043  min
12000 :	 0.37395673990249634  -i* 0.08908064

In [281]:
#Seed for reproducibility 
torch.manual_seed(1)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-4, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 2, 3
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.5184628367424011  -i* 0.6599856615066528 	 0.325
1000 :	 0.6482271552085876  -i* 0.015871914103627205 	 0.35000000000000003
2000 :	 0.5902848839759827  -i* 0.013024290092289448 	 0.37500000000000006
3000 :	 0.5948226451873779  -i* 0.0320114903151989 	 0.4000000000000001
4000 :	 0.6076419949531555  -i* 0.11084459722042084 	 0.4250000000000001
5000 :	 0.595758855342865  -i* 0.09588608890771866 	 0.4500000000000001
6000 :	 0.596686601638794  -i* 0.09718628972768784 	 0.47500000000000014
7000 :	 0.5995113253593445  -i* 0.0929717943072319 	 0.5000000000000001
8000 :	 0.5992416143417358  -i* 0.09322108328342438 	 0.5250000000000001
9000 :	 0.5990418195724487  -i* 0.09238383919000626 	 0.5500000000000002
10000 :	 0.6000069975852966  -i* 0.09273899346590042 	 0.5750000000000002
Eigenvalue founded!	Frequency =  0.6 - i 0.09274 	Epoch = 10000 	Time =  0.9805  min
11000 :	 0.5994181632995605  -i* 0.09288763254880905 	 0.6000000000000002
12000 :	 0.5997721552848816  -i* 0.09275585412979126 

KeyboardInterrupt: 

In [279]:
#Seed for reproducibility 
torch.manual_seed(0)
# left <x0> and right <xf> boundary points
x0 = 0 
xf = 1
# training points, neurons per layer, training epochs, learning rate, etc
n_train, neurons, epochs, lr, explore, tol, new = 100, 50, int(90e3), 8e-3, 1000, 1e-7, 1e-2  
#Mass, spin, azimuthal number
M, s, l = 1, 2, 4
model,loss_hists, FreqRe, FreqIm, runTime = Scan_QNM(x0, xf, neurons, epochs, n_train, lr, explore, tol, new, M, s, l)

0 :	 0.6292397379875183  -i* 0.5217627882957458 	 0.325
1000 :	 0.8518630266189575  -i* 0.01039290800690651 	 0.35000000000000003
2000 :	 0.8329742550849915  -i* 0.006205904763191938 	 0.37500000000000006
3000 :	 0.7961552143096924  -i* 0.007899395190179348 	 0.4000000000000001
4000 :	 0.8048325777053833  -i* 0.01664397120475769 	 0.4250000000000001
5000 :	 0.811923623085022  -i* 0.09878812730312347 	 0.4500000000000001
6000 :	 0.8139855861663818  -i* 0.08307240158319473 	 0.47500000000000014
7000 :	 0.8047277331352234  -i* 0.09436578303575516 	 0.5000000000000001
8000 :	 0.8077703714370728  -i* 0.09634348005056381 	 0.5250000000000001
9000 :	 0.8093965649604797  -i* 0.0946936085820198 	 0.5500000000000002
10000 :	 0.8100280165672302  -i* 0.09425392001867294 	 0.5750000000000002
11000 :	 0.8093999624252319  -i* 0.09494447708129883 	 0.6000000000000002
12000 :	 0.8096932172775269  -i* 0.09423871338367462 	 0.6250000000000002
Eigenvalue founded!	Frequency =  0.8097 - i 0.09424 	Epoch = 1